In [31]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

In [72]:
df=pd.read_csv('F:\VSCode\Cognizance\Resources\Task-4\Q4.csv')

In [73]:
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [74]:
df.groupby('Category').count()
# Here we can see that we have 747 spam messgaes and 4825 normal messages

,Message
Category,
ham,4825
spam,747


In [75]:
cat=pd.get_dummies(df['Category'],drop_first=True)
cat.head()

,spam
0,0
1,0
2,1
3,0
4,0


In [76]:
df=pd.concat([df,cat],axis=1)

In [77]:
df.head()
# Here we have successfully concatinated the original dataFrame with the boolean generated using Category

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [78]:
# df=df.drop(['Category'],axis=1)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [79]:
df['Message'] = df['Message'].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip()
df['Message'] = df['Message'].str.lower()
df['Message'] = df['Message'].str.split()

C:\Users\TEJASH REDDY\AppData\Local\Temp\ipykernel_3348\1604600185.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Message'] = df['Message'].str.replace('\W+', ' ').str.replace('\s+', ' ').str.strip()


In [80]:
train_data = df.sample(frac=0.8,random_state=1).reset_index(drop=True)
test_data = df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

In [81]:
df['Message'].value_counts() / df.shape[0] * 100

[sorry, i, ll, call, later]                                                                                                                                                      0.538406
[ok]                                                                                                                                                                             0.340991
[i, cant, pick, the, phone, right, now, pls, send, a, message]                                                                                                                   0.215363
[okie]                                                                                                                                                                           0.125628
[ok, lor]                                                                                                                                                                        0.089734
                                                                      

In [82]:
train_data['Message'].value_counts() / train_data.shape[0] * 100

[sorry, i, ll, call, later]                                                                                                                                                                                                                                                           0.471063
[ok]                                                                                                                                                                                                                                                                                  0.336474
[i, cant, pick, the, phone, right, now, pls, send, a, message]                                                                                                                                                                                                                        0.224316
[okie]                                                                                                                                     

In [83]:
test_data['Message'].value_counts() / test_data.shape[0] * 100

[sorry, i, ll, call, later]                                                                                                                         0.359066
[ok]                                                                                                                                                0.269300
[i, cant, pick, the, phone, right, now, pls, send, a, message]                                                                                      0.269300
[oh, k, i, will, come, tomorrow]                                                                                                                    0.179533
[camera, you, are, awarded, a, sipix, digital, camera, call, 09061221066, fromm, landline, delivery, within, 28, days]                              0.179533
                                                                                                                                                      ...   
[i, hope, your, pee, burns, tonite]                       

In [84]:
vocabulary = list(set(train_data['Message'].sum()))
word_counts_per_sms = pd.DataFrame([
    [row[1].count(word) for word in vocabulary]
    for _, row in train_data.iterrows()], columns=vocabulary)
train_data = pd.concat([train_data.reset_index(), word_counts_per_sms], axis=1).iloc[:,1:]

In [94]:
train_data.head()

,Category,Message,spam,appreciate,mel,woohoo,ne,started,087123002209am,disconnected,...,buzz,babes,box,4217,searching,tel,1000s,shut,okors,isn
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
Pspam = train_data['Category'].value_counts()['spam'] / train_data.shape[0]
# possibility of a message to be spam 

In [96]:
# Possibility of a message to be ham
Pham = train_data['Category'].value_counts()['ham'] / train_data.shape[0]

In [97]:
Nspam = train_data.loc[train_data['Category'] == 'spam', 
                       'Message'].apply(len).sum()
# NUmber of words in Spam message

In [98]:
Nham = train_data.loc[train_data['Category'] == 'ham', 
                       'Message'].apply(len).sum()
# Number of word in ham message 

In [99]:
Nvoc = len(train_data.columns)-3

In [100]:
def p_w_spam(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['Category'] == 'spam', word].sum() + alpha) / (Nspam + alpha*Nvoc)
    else:
        return 1
      
def p_w_ham(word):
    if word in train_data.columns:
        return (train_data.loc[train_data['Category'] == 'ham', word].sum() + alpha) / (Nham + alpha*Nvoc)
    else:
        return 1

In [114]:
def classify(Message):
    p_spam_given_message = Pspam
    p_ham_given_message = Pham
    for word in message:
        p_spam_given_message *= p_w_spam(word)
        p_ham_given_message *= p_w_ham(word)
    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_ham_given_message < p_spam_given_message:
        return 'spam'
    else:
        return 'needs human classification'

In [115]:
df

,Category,Message,spam
0,ham,"[go, until, jurong, point, crazy, available, o...",0
1,ham,"[ok, lar, joking, wif, u, oni]",0
2,spam,"[free, entry, in, 2, a, wkly, comp, to, win, f...",1
3,ham,"[u, dun, say, so, early, hor, u, c, already, t...",0
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,...",0
...,...,...,...
5567,spam,"[this, is, the, 2nd, time, we, have, tried, 2,...",1
5568,ham,"[will, ü, b, going, to, esplanade, fr, home]",0
5569,ham,"[pity, was, in, mood, for, that, so, any, othe...",0
5570,ham,"[the, guy, did, some, bitching, but, i, acted,...",0
